In [ ]:
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.edge.service import Service

from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import json
import os
import time

def load_cookie():
    # 加载并设置 Cookie
    if os.path.exists('cookies.json'):
        with open('cookies.json', 'r') as f:
            cookies = json.load(f)
            for cookie in cookies:
                driver.add_cookie(cookie)

        # 等待页面加载完成
        wait = WebDriverWait(driver, 10)  # 最多等10秒
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'body')))  # 等待页面的 body 元素加载完成

        # 刷新页面，查看是否成功加载 Cookie
        driver.refresh()

def save_cookie():
    cookies = driver.get_cookies()
    with open('cookies.json', 'w') as f:
        json.dump(cookies, f)
    print("Cookies 已保存！")

def login_operate():
    try:
        # 设置显式等待
        wait = WebDriverWait(driver, 10)  # 等待最多10秒
        # 使用data-testid定位输入框
        login_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '[data-testid="to_login_button"]')))

        login_btn.click()

        wait = WebDriverWait(driver, 10)
        phone_number = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '[data-testid="login_phone_number_input"]')))
        phone_number.send_keys('你的手机号')  # 替换为你的手机号

        wait = WebDriverWait(driver, 10)
        checkbox = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.semi-checkbox-inner')))
        checkbox.click()

        # 设置显式等待
        wait = WebDriverWait(driver, 10)  # 等待最多10秒
        # 使用data-testid定位输入框
        login_next_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '[data-testid="login_next_button"]')))
        login_next_btn.click()
        return True
    except Exception as e:
        return False

def check_login():
    try:
        wait = WebDriverWait(driver, 10)  # 等待最多10秒
        wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '[data-testid="chat_header_avatar_button"]')))
        return True
    except Exception as e:
        return False

def login():
    load_cookie()
    res = login_operate()
    if not res:
        save_cookie()
        return
    while True:
        time.sleep(2)
        if check_login():
            save_cookie()
            return

# 设置 Selenium 浏览器使用代理
options = Options()
# options.add_argument("--disable-features=msEdgeImport")  # 禁用 Edge 数据导入
service = Service(executable_path='edgedriver_win64/msedgedriver.exe')

# 创建一个新的 Edge 浏览器实例
driver = webdriver.Edge(options=options)

# 打开网页
driver.get("https://www.doubao.com/")
wait = WebDriverWait(driver, 10)  # 最多等10秒
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'body')))  # 等待页面的 body 元素加载完成
time.sleep(2)
login() #进行登陆操作

Cookies 已保存！


In [66]:
from selenium.webdriver.common.keys import Keys
import pyperclip
import win32clipboard
from io import BytesIO
from PIL import Image
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def copy_image_to_clipboard(image_path):
    image = Image.open(image_path)
    output = BytesIO()
    image.convert("RGB").save(output, "BMP")
    data = output.getvalue()[14:]  # BMP 文件头的前 14 字节要去掉
    output.close()

    win32clipboard.OpenClipboard()
    win32clipboard.EmptyClipboard()
    win32clipboard.SetClipboardData(win32clipboard.CF_DIB, data)
    win32clipboard.CloseClipboard()

def send_prompt():
    count=0
    while True:
        try:
            wait = WebDriverWait(driver, 5)  # 等待最多10秒
            # 发送信息按钮
            send_meg = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '[data-testid="chat_input_send_button"]')))
            send_meg.click()
            return 1
        except Exception as e:
            time.sleep(1)
            count+=1
            if count>=2:
                return None
    
            
def driver_refresh():
    time.sleep(2)
    driver.refresh()
    time.sleep(2)
    # wait = WebDriverWait(driver, 10)  # 等待最多10秒
    # wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'body')))

def create_conversation():
    try:
        wait = WebDriverWait(driver, 1)  # 等待最多10秒
        new_dialogue_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '[data-testid="create_conversation_button"]')))
        new_dialogue_btn.click()
        wait = WebDriverWait(driver, 10)  # 等待最多10秒
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'body')))
        return
    except Exception as e:
        pass
    try:
        wait = WebDriverWait(driver, 1)  # 等待最多10秒
        new_dialogue_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.create-chat-TZEIAV')))
        new_dialogue_btn.click()
        wait = WebDriverWait(driver, 10)  # 等待最多10秒
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'body')))
        return
    except Exception as e:
        pass

def check_content(file, content):
    if '无法' in content and '识别' in content:
        print(f'{os.path.basename(file)}: \t{content}')
        return False
    if '上传' in content and '图片' in content:
        print(f'{os.path.basename(file)}: \t{content}')
        return False
    if '提供' in content and '图片' in content:
        print(f'{os.path.basename(file)}: \t{content}')
        return False
    if '发送' in content and '图片' in content:
        print(f'{os.path.basename(file)}: \t{content}')
        return False
    if '提供' in content and '内容' in content:
        print(f'{os.path.basename(file)}: \t{content}')
        return False
    if '上传' in content and '图片' in content:
        print(f'{os.path.basename(file)}: \t{content}')
        return False
    if '识别' in content and '图片' in content:
        print(f'{os.path.basename(file)}: \t{content}')
        return False
    if '图中' in content and '文字' in content:
        print(f'{os.path.basename(file)}: \t{content}')
        return False
    if '模糊' in content:
        print(f'{os.path.basename(file)}: \t{content}')
        return False
    if '不清晰' in content:
        print(f'{os.path.basename(file)}: \t{content}')
        return False
    if '文字' in content and '内容' in content:
        print(f'{os.path.basename(file)}: \t{content}')
        return False
    if '潦草' in content:
        print(f'{os.path.basename(file)}: \t{content}')
        return False
    return True

def upload_file(image_path):
    if os.path.exists(image_path):
        try:
            wait = WebDriverWait(driver, 10)  # 等待最多10秒
            # 使用data-testid定位输入框
            prompt = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '[data-testid="chat_input_input"]')))
            prompt.send_keys(Keys.CONTROL + 'a')  # 选择所有内容
            prompt.send_keys(Keys.BACKSPACE)  # 删除内容
            copy_image_to_clipboard(image_path)
            prompt.send_keys(Keys.CONTROL + 'v')  # 粘贴剪贴板内容

            # wait = WebDriverWait(driver, 10)  # 等待最多10秒
            # file_input = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '[data-testid="upload-file-input"][class="hidden"][type="file"]')))
            # image_path = os.path.abspath(image_path)
            # driver.execute_script("arguments[0].value = '';", file_input)
            # file_input.send_keys(image_path)
            prompt.send_keys('以标准的LATEX格式识别这张图的内容，并且将识别的结果回复给我，除此之外不得回答任何其他的内容。如果你无法识别图片，或者图片模糊看不清，就直接回复‘图片模糊’并且不要再回复其他内容。有什么就回答什么就好,不要有任何额外的解释、说明、注释在在你的回答中。')

            wait = WebDriverWait(driver, 10)  # 等待最多10秒
            wait.until(EC.presence_of_element_located((By.XPATH, '//*[@alt="image"]')))
            if not send_prompt():
                return 1
            return 0
        except Exception as e:
            print(e)
            return 1
    else:
        print("文件不存在！")
        return 2
    
import traceback
def del_chat():
    try:
        wait = WebDriverWait(driver, 0.1)  # 等待最多10秒
        history_conversations = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '[data-testid="chat_list_item_title"]')))
        actions = ActionChains(driver)
        actions.move_to_element(history_conversations[0]).perform()

        history_conversations = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '[class="flex"][aria-haspopup="dialog"][tabindex="0"]')))
        history_conversations[0].click()

        del_history_conversation = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '[role="menuitem"][tabindex="-1"][aria-disabled="false"][class="remove-btn-oCMCwy select-none semi-dropdown-item"]')))
        del_history_conversation.click()

        del_comfirm_btn = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '[aria-label="confirm"][type="button"][aria-disabled="false"]')))
        del_comfirm_btn.click()
        return
    except Exception as e:
        del_chat()

import re
def copy_reply():
    first_time = time.time()
    while True:
        try:
            like_btns = WebDriverWait(driver, 18).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '[data-testid="message_action_like"]')))
            # 创建 ActionChains 对象并执行悬停操作

            copy_btns = WebDriverWait(driver, 2).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '[data-testid="message_action_copy"]')))
            action = ActionChains(driver)
            action.move_to_element(copy_btns[-1]).click().perform()
            
            return pyperclip.paste()
        
        except Exception as e:
            cur_time = time.time()
            if cur_time - first_time > 61:
                return None
            time.sleep(0.2)

In [ ]:
import os
import re
import random

image_folder_path = '集合专项/images'
image_name_list = [f for f in os.listdir(image_folder_path) if f.lower().endswith('jpg')]
image_name_list.sort(key=lambda x: int(re.search(r'(\d+)', x).group()))
image_path_list = [os.path.join(image_folder_path, f) for f in image_name_list]
parent_folder = os.path.dirname(image_folder_path)
label_folder = os.path.join(parent_folder, 'labels')
os.makedirs(label_folder, exist_ok=True)

def save_label(save_path, content):
    # content = re.sub(r"\n\s*\n", "\n", content)  # Replace multiple blank lines with one
    # content = content.replace('\\hill', '\\dothill')
    content = re.sub(r'```latex\r\n', '', content)
    content = re.sub(r'\r\n```', '', content)
    with open(save_path, 'w', encoding='utf-8') as f:
        f.write(content)

def delete_img_label(image_path, label_path):
    if os.path.exists(image_path):
        os.remove(image_path)
    if os.path.exists(label_path):
        os.remove(label_path)

for i, image_path in enumerate(image_path_list[10: ], start=1):
    base_name = os.path.basename(image_path)
    txt_name = base_name.split('.')[0] + '.txt'
    save_path = os.path.join(label_folder, txt_name)

    first_time = time.time()
    
    try:
        while True:
            rt_num = upload_file(image_path)
            if rt_num == 2:
                break
            if rt_num==1:
                create_conversation()
                driver_refresh()
                time.sleep(1)
                continue
            
            second_time = time.time()
            if second_time - first_time >= 120:
                delete_img_label(image_path, save_path)
                break

            content = copy_reply()
            if content:
                if isinstance(content, str) and check_content(image_path, content):
                    content = content.strip()
                    save_label(save_path, content)
                    time.sleep(0.1)
                    break
                else:
                    delete_img_label(image_path, save_path)
                    break
            else:
                create_conversation()
                driver_refresh()
                print(f'copy_reply的问题')
                time.sleep(1)
                continue

        time.sleep(random.uniform(0.2, 0.4))
        del_chat()
        # if i % 100 == 0:
        #     create_conversation()
        time.sleep(random.uniform(0.2, 0.4))
            
    finally:
        if 'www.doubao.com' not in driver.current_url:
            driver.get('https://www.doubao.com/')